In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [2]:
X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')

In [3]:
c1 = LogisticRegression(random_state= 1)
c2 = DecisionTreeClassifier(random_state= 1)
c3 = GaussianNB()

ec = VotingClassifier(estimators= [('lr', c1), ('rf', c2), ('gnb', c3)], voting= 'hard')

In [4]:
from sklearn.model_selection import cross_val_score

cross_val_score(ec, X, y, cv= 5).mean()
# 항상 앙상블 모델이 가장 좋은 성능을 내는 것이 아님
# 해당 예제에서는 세번째 모델이 성능이 아주 낮아서 앙상블의 스코어 값이 나머지 두 모델의 스코어 값보다 낮게 나옴

0.8009268075922046

In [5]:
cross_val_score(c1, X, y, cv= 5).mean()

0.8290420872214816

In [6]:
cross_val_score(c2, X, y, cv= 5).mean()

0.7829175395162826

In [7]:
cross_val_score(c3, X, y, cv= 5).mean()

0.4600139655938551

In [8]:
ec1 = VotingClassifier(estimators= [('lr', c1), ('dt', c2)], voting= 'hard')
cross_val_score(ec1, X, y, cv= 5).mean()
# 가장 성능이 안좋은 세번째 모델을 제거해서 스코어 값을 좋게 만듦

0.8222687742017394

In [21]:
c_params = [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]
params = {
    'lr__solver': ['liblinear'],
    'lr__penalty': ['l2'],
    'lr__C': c_params,
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [10, 8, 7, 6, 5, 4, 3, 2],
    'dt__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9]
}

In [22]:
from sklearn.model_selection import GridSearchCV

g = GridSearchCV(estimator= ec1, param_grid= params, cv= 5)
g.fit(X, y)
g.best_score_

# 투표 분류기에 들어갈 모델들의 하이퍼 매개변수를 조정하여 앙상블 스코어를 더 좋게 만들 수 있다
# 교차 검증을 통해 가장 좋은 스코어를 내는 파라미터들을 찾는다

0.8425569732749316

In [23]:
g.best_params_

{'dt__criterion': 'gini',
 'dt__max_depth': 10,
 'dt__min_samples_leaf': 5,
 'lr__C': 5.0,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}

In [28]:
# 찾은 파라미터들을 투표 분류기에 넣을 모델에 직접 설정하면 좋은 스코어를 낼 수 있다.
c1 = LogisticRegression(solver= 'liblinear', penalty= 'l2', C= 5.0, random_state= 1)
c2 = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, min_samples_leaf= 5, random_state= 1)

ec3 = VotingClassifier(estimators= [('A', c1), ('B', c2)], voting= 'hard')

cross_val_score(ec3, X, y, cv= 5).mean()

0.8425569732749316

### 정리 조합

In [30]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')

c1 = LogisticRegression(solver= 'liblinear', penalty= 'l2', C= 5.0, random_state= 1)
c2 = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, min_samples_leaf= 5, random_state= 1)

ec = VotingClassifier(estimators= [('c1', c1), ('c2', c2)], voting= 'hard')

cross_val_score(ec, X, y, cv= 5).mean()

0.8425569732749316